# 如何使用输出修复解析器

这个[输出解析器](/docs/concepts/output_parsers/)包装了另一个输出解析器，如果第一个解析器失败，它会调用另一个 LLM 来修复任何错误。

但除了抛出错误之外，我们还可以做其他事情。具体来说，我们可以将格式错误的输出连同格式化指令一起传递给模型，并要求它进行修复。

在这个示例中，我们将使用上面的 Pydantic 输出解析器。让我们看看如果我们传入一个不符合模式的结果会发生什么：

In [1]:
from typing import List

from langchain_core.exceptions import OutputParserException
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

In [ ]:
class Actor(BaseModel):
    name: str = Field(description="演员的姓名")
    film_names: List[str] = Field(description="他们主演的电影名称列表")


actor_query = "生成一个随机演员的电影作品。"

parser = PydanticOutputParser(pydantic_object=Actor)

In [3]:
misformatted = "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

In [4]:
try:
    parser.parse(misformatted)
except OutputParserException as e:
    print(e)

Invalid json output: {'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE


现在我们可以构建并使用一个 `OutputFixingParser`。这个输出解析器接受另一个输出解析器作为参数，同时还需要一个 LLM 来尝试修正任何格式错误。

In [5]:
from langchain.output_parsers import OutputFixingParser

new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())

In [6]:
new_parser.parse(misformatted)

Actor(name='Tom Hanks', film_names=['Forrest Gump'])

查看 [OutputFixingParser](https://python.langchain.com/api_reference/langchain/output_parsers/langchain.output_parsers.fix.OutputFixingParser.html#langchain.output_parsers.fix.OutputFixingParser) 的 API 文档。